In [1]:
%load_ext autoreload
import numpy as np
import matplotlib.pyplot as plt
from pandas_ml_utils import pd

In [2]:
df = pd.read_csv("SPY.csv", parse_dates=True)

In [22]:
from pandas_ml_utils import PostProcessedFeaturesAndLabels, FittingParameter
from pandas_ml_common.sampling.splitter import naive_splitter
from pandas_ml_utils_torch import PytorchNN, PytorchModel, PytorchNNFactory, LambdaSplitter
from pandas_ml_utils_torch.distribution import Logistic
from pandas_ml_utils_torch.loss import DistributionNLL
from torch.distributions import MixtureSameFamily, Normal, Categorical
from torch.optim import Adam
from torch import nn
import torch as t

factory = PytorchNNFactory.create(
    nn.Sequential(
        nn.Linear(10, 20),
        nn.Tanh(),
        nn.Linear(20, 20),
        nn.Tanh(),
        nn.Linear(20, 5),
        LambdaSplitter(
            lambda x: t.sigmoid(x[...,[0]]),
            lambda x: t.exp(x[...,1:3]),
            lambda x: x[...,3:5]
        )
    ),
    predictor=lambda n, i: t.cat(n(i), dim=1),
    trainer=lambda n, i: n(i)
)

print(Logistic(loc=t.randn(10), scale=t.ones(10)).log_prob(t.ones(10)))

cat = t.rand(10, 1)

dist = MixtureSameFamily(
    Categorical(probs=t.cat([1-cat, cat], dim=1)),
    Logistic(loc=t.randn(10, 2), scale=t.ones(10, 2))
)

print(dist.sample())


def loss(y_pred):
    probs, scales, locs = y_pred
    probs = t.cat((1-probs, probs), dim=1)
    # print(probs.shape, scales.shape, locs.shape)

    return MixtureSameFamily(
        Categorical(probs=probs),
        Logistic(loc=locs, scale=scales)
    )


print(factory().training)
print(factory()(t.randn(2, 10)))
print(DistributionNLL(loss)(factory()(t.randn(2, 10)), t.randn(2, 1)))
# print(loss( factory()(t.randn(2, 10)) ) )

tensor([-1.5747, -1.7265, -1.6915, -1.6334, -1.6490, -1.5413, -1.3870, -1.4020,
        -1.3863, -3.4128])


IndexError: tuple index out of range

In [6]:
import pandas_ta_quant
import pandas_ml_quant

df.model.fit(
    PytorchModel(
        factory,
        PostProcessedFeaturesAndLabels(
            features=[lambda df: df["Close"].pct_change()],
            feature_post_processor=lambda df: df.ta.rnn(10),
            labels=[lambda df: df["Close"].pct_change()],
        ),
        lambda: DistributionNLL(loss),
        Adam,
    ),
    FittingParameter(epochs=10, splitter=naive_splitter())
)



In [ ]:
with df.model('/tmp/mixture_probabilistic.model') as m:
    from pandas_ml_utils import FeaturesAndLabels, FittingParameter
    from pandas_ml_utils_torch import PytorchNN, PytorchModel, PytorchNNFactory, LambdaSplitter
    from pandas_ml_common.sampling.splitter import dummy_splitter
    from torch.optim import Adam
    from torch import nn
    import torch as t

    factory = PytorchNNFactory.create(
        nn.Sequential(
            nn.Linear(10, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 3),
            LambdaSplitter(
                lambda x: t.sigmoid(x[...,0]),
                lambda x: t.exp(x[...,1]),
                lambda x: x[...,2]
            )
        )
    )

    print(factory().forward(t.randn(2, 10)))

In [ ]:
    fit = m.fit(
        PytorchModel(
            factory,
            FeaturesAndLabels(
                ["x"],
                ["y"],
            ),
            HeteroscedasticityLoss,
            Adam,
        ),
        FittingParameter(epochs=2000, splitter=dummy_splitter)
    )
